 <a href="https://colab.research.google.com/github/AI4Finance-Foundation/FinRL/blob/master/FinRL_StockTrading_NeurIPS_2018.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

 # Distributed Deep Reinforcement Learning for Multiple Stock Trading

 # Content

 * [1. Problem Definition](#0)
 * [2. Getting Started - Load Python packages](#1)
     * [2.1. Install Packages](#1.1)
     * [2.2. Check Additional Packages](#1.2)
     * [2.3. Import Packages](#1.3)
     * [2.4. Create Folders](#1.4)
 * [3. Download Data](#2)
 * [4. Preprocess Data](#3)
     * [4.1. Technical Indicators](#3.1)
     * [4.2. Perform Feature Engineering](#3.2)
 * [5.Build Environment](#4)
     * [5.1. Training & Trade Data Split](#4.1)
     * [5.2. User-defined Environment](#4.2)
     * [5.3. Initialize Environment](#4.3)
 * [6.Implement DRL Algorithms](#5)
 * [7.Backtesting Performance](#6)
     * [7.1. BackTestStats](#6.1)
     * [7.2. BackTestPlot](#6.2)
     * [7.3. Baseline Stats](#6.3)
     * [7.3. Compare to Stock Market Index](#6.4)
 * [RLlib Section](#7)

 <a id='0'></a>
 # Part 1. Problem Definition

 This problem is to design an automated trading solution for single stock trading. We model the stock trading process as a Markov Decision Process (MDP). We then formulate our trading goal as a maximization problem.

 The algorithm is trained using Deep Reinforcement Learning (DRL) algorithms and the components of the reinforcement learning environment are:


 * Action: The action space describes the allowed actions that the agent interacts with the
 environment. Normally, a ∈ A includes three actions: a ∈ {−1, 0, 1}, where −1, 0, 1 represent
 selling, holding, and buying one stock. Also, an action can be carried upon multiple shares. We use
 an action space {−k, ..., −1, 0, 1, ..., k}, where k denotes the number of shares. For example, "Buy
 10 shares of AAPL" or "Sell 10 shares of AAPL" are 10 or −10, respectively

 * Reward function: r(s, a, s′) is the incentive mechanism for an agent to learn a better action. The change of the portfolio value when action a is taken at state s and arriving at new state s',  i.e., r(s, a, s′) = v′ − v, where v′ and v represent the portfolio
 values at state s′ and s, respectively

 * State: The state space describes the observations that the agent receives from the environment. Just as a human trader needs to analyze various information before executing a trade, so
 our trading agent observes many different features to better learn in an interactive environment.

 * Environment: Dow 30 consituents


 The data of the single stock that we will be using for this case study is obtained from Yahoo Finance API. The data contains Open-High-Low-Close price and volume.


 <a id='1'></a>
 # Part 2. Getting Started- Load Python Packages

 <a id='1.1'></a>
 ## 2.1. Install all the packages through FinRL library


In [2]:
!pip install git+https://github.com/AI4Finance-LLC/FinRL-Library.git

  Cloning https://github.com/AI4Finance-LLC/FinRL-Library.git to /tmp/pip-req-build-ylws0kfs
  Running command git clone -q https://github.com/AI4Finance-LLC/FinRL-Library.git /tmp/pip-req-build-ylws0kfs
  Created wheel for finrl: filename=finrl-0.3.4-py3-none-any.whl size=3885432 sha256=920880ee4e4095daebd259344287c9275b1191935c8d545c308b8a6ff32d25aa
  Stored in directory: /tmp/pip-ephem-wheel-cache-sy4lrrcy/wheels/e8/19/74/11261997d6bdca44ba73e8eeedb94a3e3d340259516a0887eb
Successfully built finrl



 <a id='1.2'></a>
 ## 2.2. Check if the additional packages needed are present, if not install them.
 * Yahoo Finance API
 * pandas
 * numpy
 * matplotlib
 * stockstats
 * OpenAI gym
 * stable-baselines
 * tensorflow
 * pyfolio

 <a id='1.3'></a>
 ## 2.3. Import Packages

In [3]:
import pandas as pd
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
# matplotlib.use('Agg')
import datetime

from apps import config
from meta.preprocessor.yahoodownloader import YahooDownloader
from meta.preprocessor.preprocessors import FeatureEngineer, data_split
from meta.env_stock_trading.env_stocktrading import StockTradingEnv
# from drl_agents.stablebaselines3.models import DRLAgent
from drl_agents.torchbeast import monobeast
from meta.data_processor import DataProcessor

from plot import backtest_stats, backtest_plot, get_daily_return, get_baseline
from pprint import pprint
from absl import flags

import sys
sys.path.append("../FinRL-Library")

import itertools

FLAGS = flags.FLAGS
FLAGS(sys.argv, known_only=True)

/home/tcai/miniconda3/envs/dwai/lib/python3.8/site-packages/setuptools/distutils_patch.py:25: UserWarning: Distutils was imported before Setuptools. This usage is discouraged and may exhibit undesirable behaviors or errors. Please use Setuptools' objects directly or at least import Setuptools first.
  warnings.warn(
/home/tcai/miniconda3/envs/dwai/lib/python3.8/site-packages/pyfolio/pos.py:26: UserWarning: Module "zipline.assets" not found; multipliers will not be applied to position notionals.
  warnings.warn(


['/home/tcai/miniconda3/envs/dwai/lib/python3.8/site-packages/ipykernel_launcher.py',
 '--ip=127.0.0.1',
 '--stdin=9008',
 '--control=9006',
 '--hb=9005',
 '--Session.signature_scheme="hmac-sha256"',
 '--Session.key=b"90677e40-427d-4af2-bedf-bec73d7a74ea"',
 '--shell=9007',
 '--transport="tcp"',
 '--iopub=9009',
 '--f=/tmp/tmp-297651RuOoJWh5nNJ.json']

 <a id='1.4'></a>
 ## 2.4. Create Folders

In [4]:
import os
if not os.path.exists("./" + config.DATA_SAVE_DIR):
    os.makedirs("./" + config.DATA_SAVE_DIR)
if not os.path.exists("./" + config.TRAINED_MODEL_DIR):
    os.makedirs("./" + config.TRAINED_MODEL_DIR)
if not os.path.exists("./" + config.TENSORBOARD_LOG_DIR):
    os.makedirs("./" + config.TENSORBOARD_LOG_DIR)
if not os.path.exists("./" + config.RESULTS_DIR):
    os.makedirs("./" + config.RESULTS_DIR)

 <a id='2'></a>
 # Part 3. Download Data
 Yahoo Finance is a website that provides stock data, financial news, financial reports, etc. All the data provided by Yahoo Finance is free.
 * FinRL uses a class **YahooDownloader** to fetch data from Yahoo Finance API
 * Call Limit: Using the Public API (without authentication), you are limited to 2,000 requests per hour per IP (or up to a total of 48,000 requests a day).




 -----
 class YahooDownloader:
     Provides methods for retrieving daily stock data from
     Yahoo Finance API

     Attributes
     ----------
         start_date : str
             start date of the data (modified from config.py)
         end_date : str
             end date of the data (modified from config.py)
         ticker_list : list
             a list of stock tickers (modified from config.py)

     Methods
     -------
     fetch_data()
         Fetches data from yahoo API


In [5]:
# from config.py start_date is a string
config.START_DATE


'2009-01-01'

In [6]:
# from config.py end_date is a string
config.END_DATE


'2021-10-31'

In [7]:
print(config.DOW_30_TICKER)


['AXP', 'AMGN', 'AAPL', 'BA', 'CAT', 'CSCO', 'CVX', 'GS', 'HD', 'HON', 'IBM', 'INTC', 'JNJ', 'KO', 'JPM', 'MCD', 'MMM', 'MRK', 'MSFT', 'NKE', 'PG', 'TRV', 'UNH', 'CRM', 'VZ', 'V', 'WBA', 'WMT', 'DIS', 'DOW']


In [8]:
data_filename = 'processed_data.csv'
data_path = os.path.join(config.DATA_SAVE_DIR, data_filename)

if os.path.exists(data_path):
    data = pd.read_csv(data_path)
else:
    data = None


In [9]:
if data is None:
    df = YahooDownloader(start_date = '2009-01-01',
                     end_date = '2021-10-31',
                     ticker_list = config.DOW_30_TICKER).fetch_data()


In [10]:
if data is None: df.shape


In [11]:
if data is None: df.sort_values(['date','tic'],ignore_index=True).head()


 # Part 4: Preprocess Data
 Data preprocessing is a crucial step for training a high quality machine learning model. We need to check for missing data and do feature engineering in order to convert the data into a model-ready state.
 * Add technical indicators. In practical trading, various information needs to be taken into account, for example the historical stock prices, current holding shares, technical indicators, etc. In this article, we demonstrate two trend-following technical indicators: MACD and RSI.
 * Add turbulence index. Risk-aversion reflects whether an investor will choose to preserve the capital. It also influences one's trading strategy when facing different market volatility level. To control the risk in a worst-case scenario, such as financial crisis of 2007–2008, FinRL employs the financial turbulence index that measures extreme asset price fluctuation.

In [12]:
if data is None:
    fe = FeatureEngineer(
                        use_technical_indicator=True,
                        tech_indicator_list = config.TECHNICAL_INDICATORS_LIST,
                        use_vix=True,
                        use_turbulence=True,
                        user_defined_feature = False)

    processed = fe.preprocess_data(df)


In [13]:
if data is None:
    list_ticker = processed["tic"].unique().tolist()
    list_date = list(pd.date_range(processed['date'].min(),processed['date'].max()).astype(str))
    combination = list(itertools.product(list_date,list_ticker))

    processed_full = pd.DataFrame(combination,columns=["date","tic"]).merge(processed,on=["date","tic"],how="left")
    processed_full = processed_full[processed_full['date'].isin(processed['date'])]
    processed_full = processed_full.sort_values(['date','tic'])

    processed_full = processed_full.fillna(0)
    processed_full.to_csv(data_path, index=False)
    
else:
    processed_full = data


In [14]:
processed_full.sort_values(['date','tic'],ignore_index=True).head(10)


,date,tic,open,high,low,close,volume,day,macd,boll_ub,boll_lb,rsi_30,cci_30,dx_30,close_30_sma,close_60_sma,vix,turbulence
0,2008-12-31,AAPL,3.070357,3.133571,3.047857,2.613433,607541200.0,2.0,0.0,2.929945,2.462269,100.0,66.666667,100.0,2.613433,2.613433,40.0,0.0
1,2008-12-31,AMGN,57.110001,58.220001,57.060001,44.657005,6287200.0,2.0,0.0,2.929945,2.462269,100.0,66.666667,100.0,44.657005,44.657005,40.0,0.0
2,2008-12-31,AXP,17.969999,18.750000,17.910000,14.950778,9625600.0,2.0,0.0,2.929945,2.462269,100.0,66.666667,100.0,14.950778,14.950778,40.0,0.0
3,2008-12-31,BA,41.590000,43.049999,41.500000,32.005886,5443100.0,2.0,0.0,2.929945,2.462269,100.0,66.666667,100.0,32.005886,32.005886,40.0,0.0
4,2008-12-31,CAT,43.700001,45.099998,43.700001,30.925043,6277400.0,2.0,0.0,2.929945,2.462269,100.0,66.666667,100.0,30.925043,30.925043,40.0,0.0
5,2008-12-31,CRM,7.712500,8.130000,7.707500,8.002500,5367600.0,2.0,0.0,2.929945,2.462269,100.0,66.666667,100.0,8.002500,8.002500,40.0,0.0
6,2008-12-31,CSCO,16.180000,16.549999,16.120001,11.868505,37513700.0,2.0,0.0,2.929945,2.462269,100.0,66.666667,100.0,11.868505,11.868505,40.0,0.0
7,2008-12-31,CVX,72.900002,74.629997,72.900002,44.129253,9964300.0,2.0,0.0,2.929945,2.462269,100.0,66.666667,100.0,44.129253,44.129253,40.0,0.0
8,2008-12-31,DIS,22.570000,22.950001,22.520000,19.538343,9012100.0,2.0,0.0,2.929945,2.462269,100.0,66.666667,100.0,19.538343,19.538343,40.0,0.0
9,2008-12-31,GS,82.239998,86.150002,81.120003,69.632233,14894100.0,2.0,0.0,2.929945,2.462269,100.0,66.666667,100.0,69.632233,69.632233,40.0,0.0


 <a id='4'></a>
 # Part 5. Design Environment
 Considering the stochastic and interactive nature of the automated stock trading tasks, a financial task is modeled as a **Markov Decision Process (MDP)** problem. The training process involves observing stock price change, taking an action and reward's calculation to have the agent adjusting its strategy accordingly. By interacting with the environment, the trading agent will derive a trading strategy with the maximized rewards as time proceeds.

 Our trading environments, based on OpenAI Gym framework, simulate live stock markets with real market data according to the principle of time-driven simulation.

 The action space describes the allowed actions that the agent interacts with the environment. Normally, action a includes three actions: {-1, 0, 1}, where -1, 0, 1 represent selling, holding, and buying one share. Also, an action can be carried upon multiple shares. We use an action space {-k,…,-1, 0, 1, …, k}, where k denotes the number of shares to buy and -k denotes the number of shares to sell. For example, "Buy 10 shares of AAPL" or "Sell 10 shares of AAPL" are 10 or -10, respectively. The continuous action space needs to be normalized to [-1, 1], since the policy is defined on a Gaussian distribution, which needs to be normalized and symmetric.

 ## Training data split: 2009-01-01 to 2020-07-01
 ## Trade data split: 2020-07-01 to 2021-10-31

In [15]:
train = data_split(processed_full, '2009-01-01','2020-07-01')
trade = data_split(processed_full, '2020-07-01','2021-10-31')
print(len(train))
print(len(trade))


83897
9744


In [16]:
train.tail()


,date,tic,open,high,low,close,volume,day,macd,boll_ub,boll_lb,rsi_30,cci_30,dx_30,close_30_sma,close_60_sma,vix,turbulence
2892,2020-06-30,UNH,288.570007,296.450012,287.660004,288.628387,2932900.0,1.0,-0.019538,304.825269,272.053921,52.413047,-25.815270,1.846804,288.873005,281.832984,30.43,12.918808
2892,2020-06-30,V,191.490005,193.750000,190.160004,191.412445,9040100.0,1.0,1.052501,199.454085,185.696446,53.021038,-51.516695,2.013358,192.162886,182.320814,30.43,12.918808
2892,2020-06-30,VZ,54.919998,55.290001,54.360001,50.990158,17414800.0,1.0,-0.442433,54.574965,49.322678,48.097046,-50.972323,8.508886,51.633274,52.091341,30.43,12.918808
2892,2020-06-30,WBA,42.119999,42.580002,41.759998,39.892540,4782100.0,1.0,-0.085829,43.544545,37.287958,48.830190,-14.445134,1.500723,39.994175,39.789724,30.43,12.918808
2892,2020-06-30,WMT,119.220001,120.129997,118.540001,116.994621,6836400.0,1.0,-0.893233,120.371803,114.363676,48.159675,-69.914656,3.847271,118.672997,120.623193,30.43,12.918808


In [17]:
trade.head()


,date,tic,open,high,low,close,volume,day,macd,boll_ub,boll_lb,rsi_30,cci_30,dx_30,close_30_sma,close_60_sma,vix,turbulence
0,2020-07-01,AAPL,91.279999,91.839996,90.977501,90.151405,110737200.0,2.0,3.022879,92.953798,80.400063,62.807135,107.487530,29.730532,84.164183,77.930893,28.620001,53.06823
0,2020-07-01,AMGN,235.520004,256.230011,232.580002,244.159119,6575800.0,2.0,3.697037,236.273225,203.552000,61.279633,271.769133,46.806139,218.441969,219.532869,28.620001,53.06823
0,2020-07-01,AXP,95.250000,96.959999,93.639999,92.347778,3301000.0,2.0,-0.390267,110.737387,88.008446,48.504819,-66.334508,3.142448,97.520673,90.952973,28.620001,53.06823
0,2020-07-01,BA,185.880005,190.610001,180.039993,180.320007,49036700.0,2.0,5.443193,220.721139,160.932863,50.925771,24.220608,15.932920,176.472335,155.614168,28.620001,53.06823
0,2020-07-01,CAT,129.380005,129.399994,125.879997,121.818474,2807800.0,2.0,1.284937,131.887567,114.449388,52.865416,35.546899,14.457404,120.567699,114.745772,28.620001,53.06823


In [18]:
config.TECHNICAL_INDICATORS_LIST


['macd',
 'boll_ub',
 'boll_lb',
 'rsi_30',
 'cci_30',
 'dx_30',
 'close_30_sma',
 'close_60_sma']

In [19]:
stock_dimension = len(train.tic.unique())
state_space = 1 + 2*stock_dimension + len(config.TECHNICAL_INDICATORS_LIST)*stock_dimension
print(f"Stock Dimension: {stock_dimension}, State Space: {state_space}")



Stock Dimension: 29, State Space: 291


In [20]:
env_kwargs = {
    "hmax": 100, 
    "initial_amount": 1000000, 
    "buy_cost_pct": 0.001,
    "sell_cost_pct": 0.001,
    "state_space": state_space, 
    "stock_dim": stock_dimension, 
    "tech_indicator_list": config.TECHNICAL_INDICATORS_LIST, 
    "action_space": stock_dimension, 
    "reward_scaling": 1e-4
    
}

e_train_gym = StockTradingEnv(df = train, **env_kwargs)


 ## Environment for Training



In [21]:
env_train, _ = e_train_gym.get_sb_env()
print(type(env_train))


<class 'stable_baselines3.common.vec_env.dummy_vec_env.DummyVecEnv'>


In [22]:
monobeast.CustomEnv.env = e_train_gym
monobeast.train(FLAGS)


[DEBUG:31296 cmd:869 2022-01-08 22:23:28,309] Popen(['git', 'version'], cwd=/mnt/e/School/KTH/Y2P2/ID2223-Scalable-ML/labs/project, universal_newlines=False, shell=None, istream=None)
[DEBUG:31296 cmd:869 2022-01-08 22:23:28,356] Popen(['git', 'version'], cwd=/mnt/e/School/KTH/Y2P2/ID2223-Scalable-ML/labs/project, universal_newlines=False, shell=None, istream=None)
[DEBUG:31296 cmd:869 2022-01-08 22:23:28,438] Popen(['git', 'cat-file', '--batch-check'], cwd=/mnt/e/School/KTH/Y2P2/ID2223-Scalable-ML/labs, universal_newlines=False, shell=None, istream=<valid stream>)
[DEBUG:31296 cmd:869 2022-01-08 22:23:28,575] Popen(['git', 'diff', '--cached', '--abbrev=40', '--full-index', '--raw'], cwd=/mnt/e/School/KTH/Y2P2/ID2223-Scalable-ML/labs, universal_newlines=False, shell=None, istream=None)
[DEBUG:31296 cmd:869 2022-01-08 22:23:28,727] Popen(['git', 'diff', '--abbrev=40', '--full-index', '--raw'], cwd=/mnt/e/School/KTH/Y2P2/ID2223-Scalable-ML/labs, universal_newlines=False, shell=None, istr

ActorNet(
  (net_state): Sequential(
    (0): Linear(in_features=291, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
  )
  (net_policy): Sequential(
    (0): Linear(in_features=542, out_features=512, bias=True)
    (1): Hardswish()
    (2): Linear(in_features=512, out_features=29, bias=True)
  )
  (net_action_std): Linear(in_features=542, out_features=29, bias=True)
  (baseline): Linear(in_features=542, out_features=1, bias=True)
)


[INFO:31296 monobeast:422 2022-01-08 22:23:30,364] # Step	total_loss	mean_episode_return	pg_loss	baseline_loss	entropy_loss
[INFO:1497 monobeast:146 2022-01-08 22:23:30,357] Actor 0 started.
[INFO:1499 monobeast:146 2022-01-08 22:23:30,377] Actor 2 started.
[INFO:1500 monobeast:146 2022-01-08 22:23:30,394] Actor 3 started.
[INFO:1498 monobeast:146 2022-01-08 22:23:30,400] Actor 1 started.
[ERROR:1497 monobeast:196 2022-01-08 22:23:30,420] Exception in worker process 0
[ERROR:1499 monobeast:196 2022-01-08 22:23:30,441] Exception in worker process 2
Traceback (most recent call last):
  File "/mnt/e/School/KTH/Y2P2/ID2223-Scalable-ML/labs/project/drl_agents/torchbeast/torchbeast/monobeast.py", line 165, in act
    buffers[key][index][0, ...] = agent_output[key]
RuntimeError: expand(torch.FloatTensor{[29]}, size=[]): the number of sizes provided (0) must be greater or equal to the number of dimensions in the tensor (1)


Process ForkProcess-1:
Traceback (most recent call last):
[ERROR:1500 monobeast:196 2022-01-08 22:23:30,457] Exception in worker process 3
Traceback (most recent call last):
  File "/mnt/e/School/KTH/Y2P2/ID2223-Scalable-ML/labs/project/drl_agents/torchbeast/torchbeast/monobeast.py", line 165, in act
    buffers[key][index][0, ...] = agent_output[key]
[ERROR:1498 monobeast:196 2022-01-08 22:23:30,459] Exception in worker process 1
  File "/home/tcai/miniconda3/envs/dwai/lib/python3.8/multiprocessing/process.py", line 315, in _bootstrap
    self.run()
RuntimeError: expand(torch.FloatTensor{[29]}, size=[]): the number of sizes provided (0) must be greater or equal to the number of dimensions in the tensor (1)
  File "/home/tcai/miniconda3/envs/dwai/lib/python3.8/multiprocessing/process.py", line 108, in run
    self._target(*self._args, **self._kwargs)


Process ForkProcess-3:
  File "/mnt/e/School/KTH/Y2P2/ID2223-Scalable-ML/labs/project/drl_agents/torchbeast/torchbeast/monobeast.py", line 199, in act
    raise e
Traceback (most recent call last):
Traceback (most recent call last):
  File "/mnt/e/School/KTH/Y2P2/ID2223-Scalable-ML/labs/project/drl_agents/torchbeast/torchbeast/monobeast.py", line 165, in act
    buffers[key][index][0, ...] = agent_output[key]
  File "/mnt/e/School/KTH/Y2P2/ID2223-Scalable-ML/labs/project/drl_agents/torchbeast/torchbeast/monobeast.py", line 165, in act
    buffers[key][index][0, ...] = agent_output[key]
  File "/mnt/e/School/KTH/Y2P2/ID2223-Scalable-ML/labs/project/drl_agents/torchbeast/torchbeast/monobeast.py", line 165, in act
    buffers[key][index][0, ...] = agent_output[key]
RuntimeError: expand(torch.FloatTensor{[29]}, size=[]): the number of sizes provided (0) must be greater or equal to the number of dimensions in the tensor (1)
Traceback (most recent call last):
RuntimeError: expand(torch.Float

  File "/home/tcai/miniconda3/envs/dwai/lib/python3.8/multiprocessing/process.py", line 315, in _bootstrap
    self.run()


  File "/home/tcai/miniconda3/envs/dwai/lib/python3.8/multiprocessing/process.py", line 108, in run
    self._target(*self._args, **self._kwargs)
Process ForkProcess-4:
Process ForkProcess-2:
  File "/mnt/e/School/KTH/Y2P2/ID2223-Scalable-ML/labs/project/drl_agents/torchbeast/torchbeast/monobeast.py", line 199, in act
    raise e
  File "/mnt/e/School/KTH/Y2P2/ID2223-Scalable-ML/labs/project/drl_agents/torchbeast/torchbeast/monobeast.py", line 165, in act
    buffers[key][index][0, ...] = agent_output[key]
Traceback (most recent call last):
Traceback (most recent call last):
RuntimeError: expand(torch.FloatTensor{[29]}, size=[]): the number of sizes provided (0) must be greater or equal to the number of dimensions in the tensor (1)
  File "/home/tcai/miniconda3/envs/dwai/lib/python3.8/multiprocessing/process.py", line 315, in _bootstrap
    self.run()
  File "/home/tcai/miniconda3/envs/dwai/lib/python3.8/multiprocessing/process.py", line 315, in _bootstrap
    self.run()
  File "/home/

ActorNet(
  (net_state): Sequential(
    (0): Linear(in_features=291, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
  )
  (net_policy): Sequential(
    (0): Linear(in_features=542, out_features=512, bias=True)
    (1): Hardswish()
    (2): Linear(in_features=512, out_features=29, bias=True)
  )
  (net_action_std): Linear(in_features=542, out_features=29, bias=True)
  (baseline): Linear(in_features=542, out_features=1, bias=True)
)


[INFO:31296 monobeast:498 2022-01-08 22:23:35,390] Steps 0 @ 0.0 SPS. Loss inf. Stats:
{}
[INFO:31296 monobeast:498 2022-01-08 22:23:40,396] Steps 0 @ 0.0 SPS. Loss inf. Stats:
{}
[INFO:31296 monobeast:498 2022-01-08 22:23:45,403] Steps 0 @ 0.0 SPS. Loss inf. Stats:
{}
[INFO:31296 monobeast:498 2022-01-08 22:23:50,410] Steps 0 @ 0.0 SPS. Loss inf. Stats:
{}
[INFO:31296 monobeast:498 2022-01-08 22:23:55,418] Steps 0 @ 0.0 SPS. Loss inf. Stats:
{}
[INFO:31296 monobeast:498 2022-01-08 22:24:00,425] Steps 0 @ 0.0 SPS. Loss inf. Stats:
{}


 ## Trading
 Assume that we have $1,000,000 initial capital at 2020-07-01. We use the DDPG model to trade Dow jones 30 stocks.

 ### Set turbulence threshold
 Set the turbulence threshold to be greater than the maximum of insample turbulence data, if current turbulence index is greater than the threshold, then we assume that the current market is volatile

In [23]:
data_risk_indicator = processed_full[(processed_full.date<'2020-07-01') & (processed_full.date>='2009-01-01')]
insample_risk_indicator = data_risk_indicator.drop_duplicates(subset=['date'])


In [24]:
insample_risk_indicator.vix.describe()


count    2893.000000
mean       18.824245
std         8.489311
min         9.140000
25%        13.330000
50%        16.139999
75%        21.309999
max        82.690002
Name: vix, dtype: float64

In [25]:
insample_risk_indicator.vix.quantile(0.996)


57.40400183105453

In [26]:
insample_risk_indicator.turbulence.describe()


count    2893.000000
mean       34.574231
std        43.787144
min         0.000000
25%        14.965867
50%        24.123804
75%        39.162151
max       652.503472
Name: turbulence, dtype: float64

In [27]:
insample_risk_indicator.turbulence.quantile(0.996)


276.44987310008247

 ### Trade

 DRL model needs to update periodically in order to take full advantage of the data, ideally we need to retrain our model yearly, quarterly, or monthly. We also need to tune the parameters along the way, in this notebook I only use the in-sample data from 2009-01 to 2020-07 to tune the parameters once, so there is some alpha decay here as the length of trade date extends.

 Numerous hyperparameters – e.g. the learning rate, the total number of samples to train on – influence the learning process and are usually determined by testing some variations.

In [28]:
#trade = data_split(processed_full, '2020-07-01','2021-10-31')
e_trade_gym = StockTradingEnv(df = trade, turbulence_threshold = 70,risk_indicator_col='vix', **env_kwargs)
# env_trade, obs_trade = e_trade_gym.get_sb_env()


In [29]:
trade.head()


,date,tic,open,high,low,close,volume,day,macd,boll_ub,boll_lb,rsi_30,cci_30,dx_30,close_30_sma,close_60_sma,vix,turbulence
0,2020-07-01,AAPL,91.279999,91.839996,90.977501,90.151405,110737200.0,2.0,3.022879,92.953798,80.400063,62.807135,107.487530,29.730532,84.164183,77.930893,28.620001,53.06823
0,2020-07-01,AMGN,235.520004,256.230011,232.580002,244.159119,6575800.0,2.0,3.697037,236.273225,203.552000,61.279633,271.769133,46.806139,218.441969,219.532869,28.620001,53.06823
0,2020-07-01,AXP,95.250000,96.959999,93.639999,92.347778,3301000.0,2.0,-0.390267,110.737387,88.008446,48.504819,-66.334508,3.142448,97.520673,90.952973,28.620001,53.06823
0,2020-07-01,BA,185.880005,190.610001,180.039993,180.320007,49036700.0,2.0,5.443193,220.721139,160.932863,50.925771,24.220608,15.932920,176.472335,155.614168,28.620001,53.06823
0,2020-07-01,CAT,129.380005,129.399994,125.879997,121.818474,2807800.0,2.0,1.284937,131.887567,114.449388,52.865416,35.546899,14.457404,120.567699,114.745772,28.620001,53.06823


In [30]:
monobeast.CustomEnv.env = e_trade_gym
df_account_value = monobeast.test(FLAGS)


ActorNet(
  (net_state): Sequential(
    (0): Linear(in_features=291, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
  )
  (net_policy): Sequential(
    (0): Linear(in_features=542, out_features=512, bias=True)
    (1): Hardswish()
    (2): Linear(in_features=512, out_features=29, bias=True)
  )
  (net_action_std): Linear(in_features=542, out_features=29, bias=True)
  (baseline): Linear(in_features=542, out_features=1, bias=True)
)


FileNotFoundError: [Errno 2] No such file or directory: '/home/tcai/logs/torchbeast/torchbeast-20220108-222328/model.tar'

In [31]:
df_account_value.shape


NameError: name 'df_account_value' is not defined

In [ ]:
df_account_value.tail()


 <a id='6'></a>
 # Part 7: Backtest Our Strategy
 Backtesting plays a key role in evaluating the performance of a trading strategy. Automated backtesting tool is preferred because it reduces the human error. We usually use the Quantopian pyfolio package to backtest our trading strategies. It is easy to use and consists of various individual plots that provide a comprehensive image of the performance of a trading strategy.

 <a id='6.1'></a>
 ## 7.1 BackTestStats
 pass in df_account_value, this information is stored in env class


In [ ]:
print("==============Get Backtest Results===========")
now = datetime.datetime.now().strftime('%Y%m%d-%Hh%M')

perf_stats_all = backtest_stats(account_value=df_account_value)
perf_stats_all = pd.DataFrame(perf_stats_all)
perf_stats_all.to_csv("./"+config.RESULTS_DIR+"/perf_stats_all_"+now+'.csv')


In [ ]:
#baseline stats
print("==============Get Baseline Stats===========")
baseline_df = get_baseline(
        ticker="^DJI", 
        start = df_account_value.loc[0,'date'],
        end = df_account_value.loc[len(df_account_value)-1,'date'])

stats = backtest_stats(baseline_df, value_col_name = 'close')



In [ ]:
df_account_value.loc[0,'date']


In [ ]:
df_account_value.loc[len(df_account_value)-1,'date']


 <a id='6.2'></a>
 ## 7.2 BackTestPlot

In [ ]:
print("==============Compare to DJIA===========")

# S&P 500: ^GSPC
# Dow Jones Index: ^DJI
# NASDAQ 100: ^NDX
backtest_plot(df_account_value, 
             baseline_ticker = '^DJI', 
             baseline_start = df_account_value.loc[0,'date'],
             baseline_end = df_account_value.loc[len(df_account_value)-1,'date'])
